In [39]:
from fastai import *
from fastai.tabular import *
import pandas as pd
from torchsummary import summary
import torch
from torch import nn
import imageio
import torch
import glob
from fastai.vision import *
import os
from torch import nn
import torch.nn.functional as F
import collections

In [4]:
colab = False

In [6]:
if colab:
    from google.colab import drive
    drive.mount('/content/drive', force_remount = True)
    %cd "/content/drive/My Drive/automatic-asset-classification"
    %ls "/content/drive/My Drive/automatic-asset-classification"
    image_path = "/content/drive/My Drive/automatic-asset-classification/data/final_dataset"
    
if not colab:
    image_path = '/Users/henriwoodcock/Documents/Code/data_projects/automatic-asset-classification/data/'\
                 'final_dataset/final'

In [15]:
size = 224
batchsize = 32
tfms = get_transforms(do_flip = False)
src = (ImageImageList.from_folder(image_path, convert_mode = 'L').split_by_rand_pct(seed=2).label_from_func(lambda 
                                                                                                            x: x))
data = (src.transform(size=size, tfm_y=True)
        .databunch(bs=batchsize)
        .normalize(imagenet_stats, do_y = False))

In [26]:
src = (ImageImageList.from_folder(image_path, convert_mode = 'L').split_by_rand_pct(seed=2).\
       label_from_func(lambda x: x, convert_mode = 'L'))

In [27]:
data = (src.transform(size=size, tfm_y=True)
        .databunch(bs=batchsize)
        .normalize(imagenet_stats, do_y = False))

In [28]:
data

ImageDataBunch;

Train: LabelList (444 items)
x: ImageImageList
Image (1, 224, 224),Image (1, 224, 224),Image (1, 224, 224),Image (1, 224, 224),Image (1, 224, 224)
y: ImageList
Image (1, 224, 224),Image (1, 224, 224),Image (1, 224, 224),Image (1, 224, 224),Image (1, 224, 224)
Path: /Users/henriwoodcock/Documents/Code/data_projects/automatic-asset-classification/data/final_dataset/final;

Valid: LabelList (110 items)
x: ImageImageList
Image (1, 224, 224),Image (1, 224, 224),Image (1, 224, 224),Image (1, 224, 224),Image (1, 224, 224)
y: ImageList
Image (1, 224, 224),Image (1, 224, 224),Image (1, 224, 224),Image (1, 224, 224),Image (1, 224, 224)
Path: /Users/henriwoodcock/Documents/Code/data_projects/automatic-asset-classification/data/final_dataset/final;

Test: None

In [112]:
class LinearAutoEncoder(nn.Module):

    def __init__(self):
        super(LinearAutoEncoder, self).__init__()
        
        self.layer_outs = [int(224*224//(6.125**i)) for i in range(6)];
        self.enc = [];
        self.dec = [];

        for i in range(len(self.layer_outs) - 1):
            self.enc += [
                nn.Linear(self.layer_outs[i], self.layer_outs[i+1]),
                nn.BatchNorm1d(self.layer_outs[i+1], eps=1e-05, momentum=0.1, affine=True, 
                               track_running_stats=True),
                nn.Tanh()
            ]
            
        self.layer_outs.reverse()
        for i in range(len(self.layer_outs) - 1):
            self.dec += [
                nn.Linear(self.layer_outs[i], self.layer_outs[i+1]),
                nn.BatchNorm1d(self.layer_outs[i+1], eps=1e-05, momentum=0.1, affine=True, 
                               track_running_stats=True),
                nn.Tanh()
            ]

        self.encoder = nn.Sequential(*self.enc)
        self.decoder = nn.Sequential(*self.dec)

        
    def encode(self, x): return self.encoder(x)
    
    def decode(self, x): return self.decoder(x)

    def forward(self, x):
        x = x.reshape([32, 224**2])
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        decoded = decoded.reshape([32,1,224,224])
        return decoded

In [113]:
autoencoder = LinearAutoEncoder();

LinearAutoEncoder(
  (encoder): Sequential(
    (0): Linear(in_features=50176, out_features=8192, bias=True)
    (1): BatchNorm1d(8192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Tanh()
    (3): Linear(in_features=8192, out_features=1337, bias=True)
    (4): BatchNorm1d(1337, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): Tanh()
    (6): Linear(in_features=1337, out_features=218, bias=True)
    (7): BatchNorm1d(218, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): Tanh()
    (9): Linear(in_features=218, out_features=35, bias=True)
    (10): BatchNorm1d(35, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Tanh()
    (12): Linear(in_features=35, out_features=5, bias=True)
    (13): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): Tanh()
  )
  (decoder): Sequential(
    (0): Linear(in_features=5, out_features=35, bias=True)
    (1): BatchNorm1d(3

In [114]:
learn = Learner(data, autoencoder, loss_func=F.mse_loss);

In [118]:
from torchsummary import summary
summary(autoencoder, (1,224,224))

RuntimeError: shape '[32, 50176]' is invalid for input of size 100352

In [8]:
class AutoEncoder(nn.Module):

    def __init__(self):
        super(AutoEncoder, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False);
        self.batch1 = nn.BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True);
        
        self.up1 = nn.Upsample(scale_factor = 2, mode = 'bilinear');
        self.conv2 = nn.Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False);
        
        self.relu = nn.ReLU(inplace = True)
        
    def encoder(self, x):
        return nn.Sequential(self.conv1, self.batch1, self.relu)(x)
    
    def decoder(self, x):
        return nn.Sequential(self.up1,self.conv2, self.relu)(x)

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [62]:
autoencoder2 = AutoEncoder();
autoencoder2

AutoEncoder(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (batch1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (up1): Upsample(scale_factor=2.0, mode=bilinear)
  (conv2): Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (relu): ReLU(inplace=True)
)

In [63]:
learn2 = Learner(data, autoencoder2, loss_func=F.mse_loss);

In [66]:
learn2.model

AutoEncoder(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (batch1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (up1): Upsample(scale_factor=2.0, mode=bilinear)
  (conv2): Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (relu): ReLU(inplace=True)
)